In [ ]:
df = df[[' Timestamp', 'AuthorName', 'DiscussionTitle', 'DiscussionBody']]

# clean the Timestamp column
df[' Timestamp'] = df[' Timestamp'].str.replace("'", "")

# clean the Timestamp column
df[' Timestamp'] = pd.to_datetime(df[' Timestamp'], format='%Y-%m-%d %H:%M:%S')


# print the first few rows to check
print(df.head())
print(df.dtypes)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from textblob import TextBlob
import pandas as pd

# load the dataset into a pandas dataframe


# clean the text data
stop_words = set(stopwords.words('english'))
df['cleaned_text'] = df['DiscussionBody'].apply(lambda x: ' '.join([word for word in x.lower().split() if word not in stop_words]))

# apply sentiment analysis using TextBlob
df['sentiment'] = df['cleaned_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

# print the overall sentiment score of each DiscussionBody
print(df[['DiscussionBody', 'sentiment']])

In [ ]:
!pip install vaderSentiment
import nltk
nltk.download('stopwords')

In [ ]:
!pip install transformer

In [ ]:
print(df.dtypes)

In [ ]:
df.head(10)

In [ ]:
import pandas as pd

# assume that your recipe data is loaded into a pandas dataframe called `recipe_df`
df.to_csv('recipe_sentiment.csv', index=False)

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
# def preprocess(text):
#     new_text = []
 
 
#     for t in text.split(" "):
#         t = '@user' if t.startswith('@') and len(t) > 1 else t
#         t = 'http' if t.startswith('http') else t
#         new_text.append(t)
#     return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
df['positive'] = 0.0
df['neutral'] = 0.0
df['negative'] = 0.0

for ind in df.index:
    print(df['DiscussionBody'][ind])
    text = df['DiscussionBody'][ind]
# text = "I dont like cars, they are accident prone"
    # text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512, add_special_tokens = True)
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)


    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
      print(f"{i+1}) {l} {np.round(float(s), 4)}")
      df[l][ind] = np.round(float(s), 4)

    # if ind > 10:
    #   break


# df.head(10)
df_result = df.groupby(by = 'DiscussionTitle', as_index=False).positive.agg('mean')
df_result
df_result_final = df_result.sort_values(by='positive', ascending=False)
df_result_final.head(25)